### IS Proof of concept - DRMABS

In [35]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import re

In [2]:
import time
from datetime import datetime
today = datetime.now()

In [3]:
import nltk
from nltk import bigrams
import collections

In [4]:
#pip install pythainlp
import pythainlp
from pythainlp import word_tokenize, Tokenizer
from pythainlp.corpus.common import thai_words
from pythainlp import sent_tokenize
from pythainlp.corpus import thai_stopwords

In [5]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [6]:
# pip install selenium
from selenium import webdriver

In [7]:
# Files definition
root_path = 'Datasource'
urls_past_file = root_path+"/urls_pasteurized.csv"
comment_past_file = root_path+"/comment_pasteurized.csv"
comment_past_nlp_file = root_path+"/comment_nlp_pasteurized.csv"

# จำนวนกระทู้ที่ต้องการให้ดึงใน 1 นาที
postNum = 6
postNumSleep = math.ceil(60/postNum)

### กรณียังไม่เคยดึง URL ที่มีการพูดถึง Keyword ใน Pantip

Get list of URL from search page

In [ ]:
urls = 'https://pantip.com/search?q=นมพาสเจอร์ไรส์'
driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
driver.get(urls)

Scroll down to get all data (Crawler)

In [ ]:
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

Extract all URLs (web scraping)

In [ ]:
content=driver.page_source
soup=BeautifulSoup(content,'lxml')
urlList = []
# ใช้ class_ เพื่อดึง div class จำเพาะ เพื่อไม่ให้ข้อความอื่นที่ไม่ต้องการปนมา
for div in soup.find_all("div", class_=lambda value: value and value=="title col-md-12"):
    for a in div.find_all("a", class_=lambda value: value and value=="datasearch-in", href=True):
        urlList.append(str(a['href']).strip())

Save URLs into csv file

In [ ]:
df_urls = pd.DataFrame(urlList)
df_urls.insert(1,'Retrived date',today)
df_urls.columns = ['URLs','Retrived date']
df_urls.to_csv(urls_past_file, index=False)

In [ ]:
driver.quit()

### กรณีเคยเก็บ URL แล้ว ให้เริ่มตรงนี้ได้เลย เพื่อไม่ให้ต้องทำ Request บ่อยๆ

Loop for scaping by URLs from file

In [8]:
df_URLRead = pd.read_csv(urls_past_file)

In [9]:
df_URLRead.head()

,URLs,Retrived date
0,https://pantip.com/topic/39702636,2020-09-22 11:08:36.425204
1,https://pantip.com/topic/39799524,2020-09-22 11:08:36.425204
2,https://pantip.com/topic/39797060,2020-09-22 11:08:36.425204
3,https://pantip.com/topic/39907475,2020-09-22 11:08:36.425204
4,https://pantip.com/topic/40142515,2020-09-22 11:08:36.425204


In [10]:
all_urls = list(df_URLRead['URLs'])

In [11]:
startResult = 0
endResult = 10

In [12]:
comments_list = []
for cnt,au in enumerate(all_urls[startResult:endResult],0):
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(au)
    content=driver.page_source
    soup=BeautifulSoup(content,'lxml')
    for div in soup.find_all("div", class_=lambda value: value and value=="display-post-story"):
        #if len(str(div.text).strip()) > 1:
        comments_list.append([au,str(div.text).strip()])
    time.sleep(postNumSleep)
    driver.quit()
    
    #จำกัดจำนวนกระทู้ สำหรับ Debug
    #if cnt == 3:
    #    break

In [13]:
df_comments = pd.DataFrame(comments_list)
df_comments.insert(2,'Retrived date',today)
df_comments.columns = ['URLs', 'text', 'Retrived date']
df_comments.to_csv(comment_past_file, index=False)

### กรณีเคย Scraping ลงไฟล์แล้ว เริ่มตรงนี้ได้เลย ป้องกันโดนแบน IP

Clean data & NLP Tokenized

In [14]:
def checkStopwords(w):
    returnList = []
    stopwords = list(thai_stopwords())
    for i in w:  # comment
        if i not in stopwords:
            returnList.append(i)
    return returnList

In [15]:
df_textList = pd.read_csv(comment_past_file)

In [16]:
def token_importantKey():
    keys = ['ซีพี','เมจิ','ดัชมิลล์','dairy home','แดรี่โฮม'
            ,'สตอเบอรี่','ช็อกโกแล็ต','คอนเฟล็ก'
            ,'พาสเจอร์ไรซ์','พลาสเจอ','พาสเจอร์ไรส์','ฟาสเจอร์'
            ,'พลาสติก','พาสติก',
            'ดัชมิลล์selected','grass fed milk']
    return keys

In [42]:
corpus = []
garbage_char = [string.punctuation,'\xa0',' ','\n']

words = set(thai_words())  # thai_words() returns frozenset
for i in token_importantKey():
    words.add(i)
custom_tokenizer = Tokenizer(words)

for ii in list(df_textList['text']):
    #wtkn = sent_tokenize(ii, engine="whitespace+newline")
    wtkn = custom_tokenizer.word_tokenize(ii)
    wtkn = [w for w in wtkn if w.isalnum()]
    wtkn = [x for x in wtkn if x not in garbage_char]
    wtkn = [y.strip('.') for y in wtkn]
    wtkn = [s for s in wtkn if not s.isdigit()]
    corpus.append(wtkn)

In [43]:
# Merge list
merge_corpus_list = list(itertools.chain.from_iterable(corpus))

In [44]:
chk_corpus_list = checkStopwords(merge_corpus_list)

In [45]:
chk_corpus_list

['รบกวน',
 'ปอ',
 'โท',
 'ทำ',
 'นม',
 'ขวด',
 'เพราะอะไร',
 'กก',
 'กก',
 'ตอบ',
 'ตอน',
 'นม',
 'พลาสเจอ',
 'ขวด',
 'นมสด',
 'นมสด',
 'ตอน',
 'วาง',
 'แถม',
 'หาย',
 'ขนาด',
 'สาย',
 'https',
 'sites',
 'google',
 'com',
 'site',
 'khorngnganthnxmxahar',
 'ทน',
 'ขวด',
 'ขนาด',
 'cc',
 'ขวด',
 'ขวด',
 'ขวด',
 'ทนทาน',
 'ขวด',
 'ขวด',
 'Cost',
 'แตก',
 'เบา',
 'ทน',
 'ทน',
 'ผม',
 'กก',
 'กก',
 'นะคะ',
 'หา',
 'คอม',
 'มอน',
 'หา',
 'คน',
 'นม',
 'ไหม',
 'ตามใจ',
 'ปาก',
 'ขนม',
 'อาหาร',
 'ขยะ',
 'ชา',
 'ขนม',
 'อาหาร',
 'อาหาร',
 'ถาม',
 'ขาย',
 'นม',
 'ขวด',
 'ไหม',
 'ขวด',
 'บาท',
 'ตก',
 'ขวด',
 'บาท',
 'ขวด',
 'บาท',
 'หรอ',
 'ขวด',
 'บาท',
 'ขวด',
 'บาท',
 'ขวด',
 'นม',
 'ขวด',
 'เจอ',
 'ไหม',
 'เมษายน',
 'เวลา',
 'อากาศ',
 'ตา',
 'ตรวจ',
 'เสมอ',
 'นม',
 'ราด',
 'ทราบ',
 'นม',
 'กาย',
 'รอ',
 'อาการ',
 'แมว',
 'แมว',
 'คาบ',
 'แมว',
 'ฝน',
 'ฝาก',
 'รอ',
 'หมอ',
 'บอ',
 'รอด',
 'หย',
 'อดนม',
 'แพะ',
 'นม',
 'นะคะ',
 'หยอด',
 'CC',
 'นม',
 'ตอน',
 'ตา',
 'รอบ',
 'ตะ',
 'หย',
 

In [46]:
df_corpus_list = pd.DataFrame(chk_corpus_list)
df_corpus_list.columns = ['text']
df_corpus_list.to_csv(comment_past_nlp_file, index=False)